### Plot data from the netCDF conversions of the original log files and compare with SSDS generated version
To run this Notebook:
- Install Python3.8 (e.g from [Anacaonda](https://www.anaconda.com/distribution/))
- `git clone https://<your_account>@bitbucket.org/mbari/auv-python.git` Replacing: `<your_account>`
- `cd auv-python`
- `jupyter notebook`
- Select this file and execute the cells

In [ ]:
import os
import xarray as xr
import hvplot.pandas
import ipywidgets as widgets
import pandas as pd
from logs2netcdfs import BASE_PATH, LOG_FILES

# Pick 'dorado' mission on local system
auv_name = 'dorado'
missions_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs')
mission_picker = widgets.Dropdown(options=os.listdir(missions_dir))
display(mission_picker)

In [ ]:
mission = mission_picker.value

# Compare SSDS generated with logs2netcdfs.py generated
ssds_path = os.path.join('http://dods.mbari.org:80/opendap/data/ssdsdata/ssds/generated/netcdf/files',
                         'ssds.shore.mbari.org/auvctd/missionlogs/', mission.split('.')[0],
                         ''.join(mission.split('.')[:2]), mission)

# Read all the netCDF file data into in-memory xarray objects saved in a dictionary
netcdfs_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs', mission)
ncs = {}
dds = {}
for lf in LOG_FILES:
    nc_file = lf.replace('.log', '.nc')
    try:
        ncs[nc_file] = xr.open_dataset(os.path.join(netcdfs_dir, nc_file))
        dds[nc_file] = xr.open_dataset(os.path.join(ssds_path, nc_file))
    except FileNotFoundError:
        pass

In [ ]:
# Print the variables in each data set
for nc_file, ds in ncs.items():
    print(f"{nc_file}: {[v for v in list(ds)]}")
    ##display(ds) # Uncomment to explore all the attributes

In [ ]:
# Pick the converted log to examine
ds_picker = widgets.Dropdown(options=ncs.keys())
display(ds_picker)

In [ ]:
# Pick the variable to plot
var_picker = widgets.Dropdown(options=list(ncs[ds_picker.value]))
display(var_picker)

In [ ]:
variable = var_picker.value

# Plot locally generated variable with the one created and served by SSDS
local_ds = ncs[ds_picker.value]
ssds_ds = dds[ds_picker.value]

var = getattr(local_ds, variable)
try:
    d_var = getattr(ssds_ds, variable)
except AttributeError as e:
    d_var = xr.DataArray([])

df = pd.concat([local_ds[variable].to_pandas().rename(f"auv-python"),
                 ssds_ds[variable].to_pandas().rename(f"SSDS")], axis=1)
display(df.hvplot(ylabel=f"{var.attrs['long_name']} [{var.attrs.get('units', '')}]"))